In [1]:
# from analyse_model import main as test
# test()
from tensor2tensor.visualization import attention
import h5py
import numpy as np
from sample_latent import visualize_latent_rep
from molecules.model import TriTransformer
from utils import AttnParams, load_properties, load_dataset, rdkit_funcs_mol
from keras.optimizers import Adam


model_dir = 'models/avg-mh8-kqv_medium_d60_VAE/'

# Load in model
model_params = AttnParams()
model_params.load(model_dir + "params.pkl")

# Prepare model
# Load the data
d_file = 'data/zinc_250k.h5'
d_type = 'cat' # should be onehot if conv or VAE
data_train, data_test, props_train, props_test, tokens = load_dataset(d_file, d_type, False)
props_train, props_test, prop_labels = load_properties(d_file)

model = TriTransformer(tokens, model_params)
model.build_models()
model.compile_vae(Adam(0.001, 0.9, 0.98, epsilon=1e-9))
model.autoencoder.load_weights(model_dir + "best_model.h5")

Using TensorFlow backend.


Using variational autoencoder


In [2]:
smiles_str = 'CC(=O)Oc1ccccc1C(=O)O'#'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O'
output_len = 10

smiles_seq = tokens.tokenize(smiles_str)
smiles_seq = [smiles_seq[0][:-1]]
output_seq = [smiles_seq[0][:output_len]]
attentions = model.output_attns.predict_on_batch([smiles_seq, output_seq])


In [3]:
l = int(len(attentions) / 3)
h = model_params["heads"]
l_in = len(smiles_seq[0])
l_out = len(output_seq[0]) - 1

reshape_attn = lambda ATT : np.swapaxes(np.expand_dims(np.array(ATT),1), 3, 4)
reshape_attn = lambda ATT : np.expand_dims(np.array(ATT),1)
enc_atts = reshape_attn(attentions[0:l])
dec_atts = reshape_attn(attentions[l:2*l])
# dec_atts = np.expand_dims(dec_atts[:,:,:,1,:],3)

encdec_atts = reshape_attn(attentions[2 * l:])
# encdec_atts = np.expand_dims(encdec_atts[:,:,:,1,:],3)

# 	enc_atts:	  (l, 1, h, in, in)
# 	dec_atts:     (l, 1, h, 1, out)
# 	encdec_atts:  (l, 1, h, 1, in)
print("Shapes of arrays:\n\tenc_atts:\t{}\n\tdec_atts:\t{}\n\tencdec_atts:\t{}".format(np.shape(enc_atts), np.shape(dec_atts), np.shape(encdec_atts)))

print("Expected shapes of arrays:\n\tenc_atts:\t{}\n\tdec_atts:\t{}\n\tencdec_atts:\t{}".format([l, 1, h, l_in, l_in], [l, 1, h, 1, l_out], [l, 1, h, 1, l_in]))

Shapes of arrays:
	enc_atts:	(3, 1, 8, 23, 23)
	dec_atts:	(3, 1, 8, 9, 9)
	encdec_atts:	(3, 1, 8, 9, 1)
Expected shapes of arrays:
	enc_atts:	[3, 1, 8, 23, 23]
	dec_atts:	[3, 1, 8, 1, 9]
	encdec_atts:	[3, 1, 8, 1, 23]


In [4]:
def call_html():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              "d3": "https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.8/d3.min",
              jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
            },
          });
        </script>
        '''))


In [5]:
in_str = [tokens.token(a) for a in smiles_seq[0]]
out_str = in_str[1:]

#out_str = list(smiles_str) + [tokens.token(tokens.endid())]
#in_str = [tokens.token(tokens.startid())] + out_str
call_html()
attention.show(in_str, out_str, enc_atts, dec_atts, encdec_atts)

ValueError: all the input array dimensions except for the concatenation axis must match exactly